In [24]:
!pip install -U deepsparse[llm] jinja2

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 133.1/133.1 kB 1.5 MB/s eta 0:00:00a 0:00:01


In [ ]:
from deepsparse import TextGeneration

model = TextGeneration(
    model="hf:mgoin/OpenHermes-2.5-Mistral-7B-pruned50-quant-ds",
    sequence_length=4096
)

In [5]:
from datasets import load_dataset

dataset = load_dataset(
    "HuggingFaceH4/ultrachat_200k",
    split="train_sft[:100]",
)

Generating test_gen split: 100%|██████████| 28304/28304 [00:00<00:00, 85248.54 examples/s]


In [27]:
MESSAGE_IDX = 93

messages = dataset["messages"][MESSAGE_IDX][:3]
print(messages)

[{'content': 'We know you’ve done some pretty awesome stuff over the past four years. Please get creative when you’re writing about those things. Honestly, when you start tackling the college essays, you should spend most of your time planning out what you’re going to write about.\nHere’s a list of yawn-worthy college essays we are just tired of reading.\nThe essay about your mission trip to Guatemala when you realized that there was poverty in the world. We’ve read this one at least 3 billion times. If you’re going to talk about your mission trip, pleeeeaseeee have a better life lesson at the end.\nThe essay about how your grandma came to the US from El Salvador or wherever and pulled herself up by her bootstraps. You have a cool grandma; that is awesome. But she’s not applying to college. Your essay needs to be about you.\nThe essay about how you went to the championships with your basketball team, and you learned that “hard work pays off”. Cool. Now let’s go about 8 zillion times de

In [28]:
system_prompt = {
    "content": "You are a helpful assistant",
    "role": "system"
}

def format_prompt(messages):
    messages = [system_prompt] + messages
    return model.tokenizer.apply_chat_template(
        messages, 
        add_generation_prompt=True, 
        tokenize=False
    )

prompt = format_prompt(messages)
print(prompt)

<|im_start|>system
You are a helpful assistant<|im_end|>
<|im_start|>user
We know you’ve done some pretty awesome stuff over the past four years. Please get creative when you’re writing about those things. Honestly, when you start tackling the college essays, you should spend most of your time planning out what you’re going to write about.
Here’s a list of yawn-worthy college essays we are just tired of reading.
The essay about your mission trip to Guatemala when you realized that there was poverty in the world. We’ve read this one at least 3 billion times. If you’re going to talk about your mission trip, pleeeeaseeee have a better life lesson at the end.
The essay about how your grandma came to the US from El Salvador or wherever and pulled herself up by her bootstraps. You have a cool grandma; that is awesome. But she’s not applying to college. Your essay needs to be about you.
The essay about how you went to the championships with your basketball team, and you learned that “hard wor

In [29]:
output = model(prompt, max_new_tokens=25)
print(output.generations[0].text)

Absolutely! Here are some alternative essay topics that can help your application stand out among others:

1. Overcoming a


In [32]:
output_iterator = model(prompt=prompt, streaming=True, max_new_tokens=100)
print(prompt, end="")
for it in output_iterator:
    print(it.generations[0].text, end="")

<|im_start|>system
You are a helpful assistant<|im_end|>
<|im_start|>user
We know you’ve done some pretty awesome stuff over the past four years. Please get creative when you’re writing about those things. Honestly, when you start tackling the college essays, you should spend most of your time planning out what you’re going to write about.
Here’s a list of yawn-worthy college essays we are just tired of reading.
The essay about your mission trip to Guatemala when you realized that there was poverty in the world. We’ve read this one at least 3 billion times. If you’re going to talk about your mission trip, pleeeeaseeee have a better life lesson at the end.
The essay about how your grandma came to the US from El Salvador or wherever and pulled herself up by her bootstraps. You have a cool grandma; that is awesome. But she’s not applying to college. Your essay needs to be about you.
The essay about how you went to the championships with your basketball team, and you learned that “hard wor